Việc phân loại tập dữ liệu và áp dụng một chức năng cho mỗi nhóm, cho dù là tổng hợp hay chuyển đổi, thường là một thành phần quan trọng của quy trình phân tích dữ liệu.Sau khi tải, hợp nhất và chuẩn bị tập dữ liệu, bạn có thể cần tính toán thống kê nhóm hoặc có thể là bảng tổng hợp cho mục đích báo cáo hoặc trực quan hóa. Pandas cung cấp 1 flexible `group by` interface

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

### 10.1 GroupBy Mechanics

Hadley Wickham, tác giả của nhiều gói phổ biến cho ngôn ngữ lập trình R, đã đặt ra thuật ngữ `split-apply-merge` để mô tả các hoạt động của `group()`. Trong giai đoạn đầu tiên của quy trình, dữ liệu chứa trong đối tượng **pandas**, dù là Series, DataFrame hay cách khác, được `split()` các nhóm dựa trên một hoặc nhiều khóa mà bạn cung cấp. Việc phân tách được thực hiện trên một trục cụ thể của một đối tượng. Ví dụ: một DataFrame có thể được nhóm trên các hàng (trục = 0) hoặc các cột của nó (trục = 1). Khi điều này được thực hiện, một hàm được áp dụng cho mỗi nhóm, tạo ra một giá trị mới. Cuối cùng, kết quả của tất cả các ứng dụng chức năng đó được `combine()` thành một đối tượng kết quả. Hình thức của đối tượng kết quả thường sẽ phụ thuộc vào những gì đang được thực hiện với dữ liệu ”

In [2]:
from IPython.display import Image
Image(url="./Images/Illustration of a group aggregation.png")

Mỗi nhóm key có thể lấy rất nhiều forms, và các key không có tất cả cùng kiểu loại

1. 1 List hoặc 1 mảng của giá trị rằng có thể cùng 1 chiều với trục đang nhóm

2. Một giá trị cho biết tên cột trong 1 Data frame

3. A dict hoặc 1 series cho bạn Đưa ra sự tương ứng giữa các giá trị trên trục được nhóm và tên nhóm"

4. A hàm có thể thực thi trên 1 axis indexx hoặc label trong index

*Lưu ý rằng ba phương pháp sau là các phím tắt để tạo ra một mảng giá trị được sử dụng để chia nhỏ đối tượng*

**Example**

In [3]:
df = pd.DataFrame({
    'key1': ['a','a','b','b','a'],
    'key2': ['one','two','one','two','one'],
    'data1': np.random.randn(5),
    'dat2': np.random.randn(5)
})

In [4]:
df

,key1,key2,data1,dat2
0,a,one,0.319909,-0.229579
1,a,two,-0.619485,0.055584
2,b,one,-0.402056,-0.049330
3,b,two,-0.053447,1.049404
4,a,one,0.937563,-1.076460


Giả sử bạn muốn tính giá trị trung bình của cột **data1** bằng cách sử dụng các nhãn từ **key1**. Có một số cách để làm điều này. Một là truy cập data1 và gọi nhóm theo cột (a Series) tại key1:

In [5]:
grouped = df['data1'].groupby(df['key1'])

In [6]:
grouped


biến `grouped ` là một **GroupBy Object**. Nó chưa thực sự tính toán bất cứ điều gì cả, nó chỉ gộp các nhóm lại với nhau thôi 

In [7]:
grouped.mean()

key1
a    0.212662
b   -0.227752
Name: data1, dtype: float64

Giờ chúng ta sẽ giải thích cái gì đã xảy ra `.mean()`. Điều quan trọng ở đây là a `Series` đã gộp lại theo group `key`, sẽ làm ra 1 new `Series mới` thực hiện gộp các index trong `key1`. Kết quả sẽ là index của `key1` đã có



Nếu thay vì chúng ta truyền nhiều array như 1 list, chúng ta sẽ truyền nhiều tham số như 1 list chúng ta sẽ đc 1 cái khác biệt

In [8]:
df

,key1,key2,data1,dat2
0,a,one,0.319909,-0.229579
1,a,two,-0.619485,0.055584
2,b,one,-0.402056,-0.049330
3,b,two,-0.053447,1.049404
4,a,one,0.937563,-1.076460


In [9]:

means = df['data1'].groupby([df['key1'],df['key2']]).mean()


In [10]:
means

key1  key2
a     one     0.628736
      two    -0.619485
b     one    -0.402056
      two    -0.053447
Name: data1, dtype: float64

now chúng ta đang gộp dữ liệu với two keys. 

In [11]:
means.unstack()

key2,one,two
key1,,
a,0.628736,-0.619485
b,-0.402056,-0.053447


Trong ví dụ này, các khóa nhóm đều là Chuỗi, mặc dù chúng có thể là bất kỳ mảng nào có độ dài phù hợp:”

In [12]:
states = np.array(['Ohio','California','California','Ohio','Ohio'])

In [13]:
years = np.array([2005,2005,2006,2005,2006])

In [14]:
df['data1'].groupby([states,years]).mean()

California  2005   -0.619485
            2006   -0.402056
Ohio        2005    0.133231
            2006    0.937563
Name: data1, dtype: float64

In [15]:
df

,key1,key2,data1,dat2
0,a,one,0.319909,-0.229579
1,a,two,-0.619485,0.055584
2,b,one,-0.402056,-0.049330
3,b,two,-0.053447,1.049404
4,a,one,0.937563,-1.076460


Thông tin nhóm thường được tìm thấy trong DataFrame giống với dữ liệu bạn muốn làm việc. Trong trường hợp đó, bạn có thể chuyển tên cột (cho dù đó là chuỗi, số hay các đối tượng Python khác) làm khóa nhóm

In [16]:
df.groupby('key1').mean()

,data1,dat2
key1,,
a,0.212662,-0.416818
b,-0.227752,0.500037


In [17]:
df

,key1,key2,data1,dat2
0,a,one,0.319909,-0.229579
1,a,two,-0.619485,0.055584
2,b,one,-0.402056,-0.049330
3,b,two,-0.053447,1.049404
4,a,one,0.937563,-1.076460


In [18]:
df.groupby(['key1','key2']).mean()

data1      dat2
key1 key2                    
a    one   0.628736 -0.653019
     two  -0.619485  0.055584
b    one  -0.402056 -0.049330
     two  -0.053447  1.049404

Group chỉ áp dụng với dữ liệu số 

`size()` để trả về kích thước tổng pt nó gộp vào 

In [19]:
df.groupby(['key1','key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

In [20]:
df.groupby('key1').size()

key1
a    3
b    2
dtype: int64

### Iterating Over Groups (Lặp các nhóm)
Đối tượng GroupBy hỗ trợ lặp lại, tạo ra một chuỗi `2 tuple` chứa `tên``nhóm` cùng với phần dữ liệu. Hãy xem xét những điều sau: ”

In [21]:
for name,group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1      dat2
0    a  one  0.319909 -0.229579
1    a  two -0.619485  0.055584
4    a  one  0.937563 -1.076460
b
  key1 key2     data1      dat2
2    b  one -0.402056 -0.049330
3    b  two -0.053447  1.049404


Trong trường hợp của nhiều key, phần tử đầu tiên trong `tuple` sẽ là 1 tuple của key xem ví dụ ở dứoi 

In [22]:
for (k1,k2), group in df.groupby(['key1','key2']):
    print((k1,k2)) # Key of group 
    print(group) # data

('a', 'one')
  key1 key2     data1      dat2
0    a  one  0.319909 -0.229579
4    a  one  0.937563 -1.076460
('a', 'two')
  key1 key2     data1      dat2
1    a  two -0.619485  0.055584
('b', 'one')
  key1 key2     data1     dat2
2    b  one -0.402056 -0.04933
('b', 'two')
  key1 key2     data1      dat2
3    b  two -0.053447  1.049404


## Selecting a Column or Subset of Columns Chọn một cột hoặc một tập hợp con các cột

indexing a Groupby Object đã tạo từ 1 DataFrame với 1 tên một cột của mảng của cột tên có hiệu ứng subsetting cho 1 kết hợp: 
Nghĩa là mình chỉ group 1 cột thôI

In [23]:
df.groupby('key1')['data1']

In [24]:
df.groupby('key1')[['dat2']] 

In [25]:
df

,key1,key2,data1,dat2
0,a,one,0.319909,-0.229579
1,a,two,-0.619485,0.055584
2,b,one,-0.402056,-0.049330
3,b,two,-0.053447,1.049404
4,a,one,0.937563,-1.076460


In [26]:
# Gộp 2 key và chỉ tính mean ở dat2 
df.groupby(['key1','key2'])[['dat2']].mean()

dat2
key1 key2          
a    one  -0.653019
     two   0.055584
b    one  -0.049330
     two   1.049404

Đối tượng được trả về bởi thao tác lập chỉ mục này là DataFrame được nhóm lại nếu một danh sách hoặc mảng được chuyển hoặc một Chuỗi được nhóm nếu chỉ một tên cột được chuyển dưới dạng một đại lượng vô hướng

In [27]:
s_grouped = df.groupby(['key1','key2'])['dat2']

In [28]:
s_grouped

In [29]:
s_grouped.mean()

key1  key2
a     one    -0.653019
      two     0.055584
b     one    -0.049330
      two     1.049404
Name: dat2, dtype: float64

### Group with Dicts and Series
Nhóm thông tin có thể tồn tại ở dạng khác với mảng.

In [30]:
people = pd.DataFrame(np.random.randn(5,5),
                         columns =['a','b','c','d','e'],
                         index = ['Joe','Steve','Wes','Jim','Travis'])

In [31]:
people

,a,b,c,d,e
Joe,-0.984333,-0.444764,0.508205,0.412094,-2.226972
Steve,0.281937,-2.488614,0.694775,2.117176,1.091318
Wes,0.427789,-0.490226,-0.430683,-2.209835,-0.301301
Jim,0.478870,-0.277060,0.916064,-0.508871,-0.068874
Travis,0.113569,0.480822,0.519937,-0.533700,0.018473


In [32]:
people.iloc[2:3,[1,2]]= np.nan

In [33]:
people

,a,b,c,d,e
Joe,-0.984333,-0.444764,0.508205,0.412094,-2.226972
Steve,0.281937,-2.488614,0.694775,2.117176,1.091318
Wes,0.427789,NaN,NaN,-2.209835,-0.301301
Jim,0.478870,-0.277060,0.916064,-0.508871,-0.068874
Travis,0.113569,0.480822,0.519937,-0.533700,0.018473


Giả sử tôi có một nhóm tương ứng cho các cột và muốn tổng hợp các cột lại với nhau theo nhóm"

In [34]:
mapping = {
    'a':'red',
    'b':'red',
    'c':'blue',
    'd':'blue',
    'e':'red',
    'f':'orange'
}

bây giờ, bạn có thể xây dựng một mảng từ dit để truyền cho groupby 

In [35]:
by_column = people.groupby(mapping,axis=1)


In [36]:
by_column

In [37]:
by_column.sum()

,blue,red
Joe,0.920299,-3.656069
Steve,2.811952,-1.115358
Wes,-2.209835,0.126488
Jim,0.407193,0.132937
Travis,-0.013764,0.612864


In [38]:
by_column = people.groupby(mapping)

In [39]:
by_column.sum()

,a,b,c,d,e


In [40]:
map_series = pd.Series(mapping)

In [41]:
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [42]:
people.groupby(map_series,axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


In [43]:
people

,a,b,c,d,e
Joe,-0.984333,-0.444764,0.508205,0.412094,-2.226972
Steve,0.281937,-2.488614,0.694775,2.117176,1.091318
Wes,0.427789,NaN,NaN,-2.209835,-0.301301
Jim,0.478870,-0.277060,0.916064,-0.508871,-0.068874
Travis,0.113569,0.480822,0.519937,-0.533700,0.018473


### Grouping with Functions 

Sử dụng các hàm Python là một cách chung chung hơn để xác định 
ánh xạ nhóm so với một dict hoặc Series. Bất kỳ hàm nào được truyền dưới dạng khóa nhóm sẽ được gọi một lần cho mỗi giá trị chỉ mục, với các giá trị trả về được sử dụng làm tên nhóm. Cụ thể hơn, hãy xem xét ví dụ DataFrame từ phần trước, có tên của mọi người làm giá trị chỉ mục. Giả sử bạn muốn nhóm theo độ dài của tên; trong khi bạn có thể tính một mảng độ dài chuỗi, đơn giản hơn là chỉ cần chuyển hàm len: ”

In [44]:
people.groupby(len).sum()

,a,b,c,d,e
3,-0.077674,-0.721824,1.424269,-2.306611,-2.597147
5,0.281937,-2.488614,0.694775,2.117176,1.091318
6,0.113569,0.480822,0.519937,-0.533700,0.018473


### Grouping by Index Levels 

    Một tiện ích cuối cùng cho các tập dữ liệu được lập chỉ mục phân cấp là khả năng tổng hợp bằng cách sử dụng một trong các cấp của chỉ mục trục. Hãy xem một ví dụ: ”


In [45]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                        [1, 3, 5, 1, 3]],
                                        names=['cty', 'tenor'])



In [46]:
columns

MultiIndex([('US', 1),
            ('US', 3),
            ('US', 5),
            ('JP', 1),
            ('JP', 3)],
           names=['cty', 'tenor'])

In [47]:
hier_df = pd.DataFrame(np.random.randn(4,5),
                      columns=columns)


In [48]:
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0     -0.956088 -0.127800  1.374452  2.016276 -0.342295
1     -1.488033 -0.599463  1.339621  1.713674  1.082617
2     -0.015296  0.345409 -1.472195 -1.066126 -0.940741
3     -0.203682  0.059312 -0.627047  0.861774  0.729942

In [49]:
hier_df.groupby(level='cty',axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


### 10.2 Data Aggregation 

In [50]:
data = {'Function name': 
            ['count',
             'sum',
             'mean',
             'median',
             'std,var',
             'min,max',
             'prod',
             'first,last'],
        'Description': [
            "Number of non-NA values in the group",
            "Sum of non-NA",
            "Mean of non-Na",
            'Median of non-Na',
            "Độ lệch chuẩn và phương sai",
            "Min và max values",
            "Product of non-Na values",
            "first and last non-Na values"
        ]
       }

In [51]:
groupby_methods = pd.DataFrame(data,columns=['Function name','Description'])

In [52]:
groupby_methods

,Function name,Description
0,count,Number of non-NA values in the group
1,sum,Sum of non-NA
2,mean,Mean of non-Na
3,median,Median of non-Na
4,"std,var",Độ lệch chuẩn và phương sai
5,"min,max",Min và max values
6,prod,Product of non-Na values
7,"first,last",first and last non-Na values


In [53]:
df

,key1,key2,data1,dat2
0,a,one,0.319909,-0.229579
1,a,two,-0.619485,0.055584
2,b,one,-0.402056,-0.049330
3,b,two,-0.053447,1.049404
4,a,one,0.937563,-1.076460


In [54]:
groued = df.groupby('key1')

In [55]:
groued['data1'].quantile(0.9)

key1
a    0.814032
b   -0.088308
Name: data1, dtype: float64

Để sử dụng function của bạn trong aggregation, truywwfn bất kỳ function nào tại araray tới thằng aggrete or `agg()`

In [56]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [57]:
groued.agg(peak_to_peak)

,data1,dat2
key1,,
a,1.557048,1.132044
b,0.348609,1.098734


Bạn có thể thất rằng các methods như `describe()` cũng hoạt động ngay cả khi chúng k gộp lại, 


In [58]:
groued.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0  0.212662  0.784045 -0.619485 -0.149788  0.319909  0.628736   
b      2.0 -0.227752  0.246504 -0.402056 -0.314904 -0.227752 -0.140599   

                dat2                                                   \
           max count      mean       std      min       25%       50%   
key1                                                                    
a     0.937563   3.0 -0.416818  0.588791 -1.07646 -0.653019 -0.229579   
b    -0.053447   2.0  0.500037  0.776922 -0.04933  0.225354  0.500037   

                          
           75%       max  
key1                      
a    -0.086997  0.055584  
b     0.774721  1.049404

## Column-Wise and multiple Function Applicaiton 

Example

In [61]:
tips = pd.read_csv("./examples/tips.csv")

In [62]:
tips

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4
...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3
240,27.18,2.00,Yes,Sat,Dinner,2
241,22.67,2.00,Yes,Sat,Dinner,2
242,17.82,1.75,No,Sat,Dinner,2


In [63]:
tips['tip_pct'] = tips['tip'] / tips['total_bill']

In [64]:
tips

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,No,Sat,Dinner,2,0.098204


. Tuy nhiên, bạn có thể muốn tổng hợp bằng một hàm khác tùy thuộc vào cột hoặc nhiều hàm cùng một lúc.

In [65]:
grouped = tips.groupby(['day','smoker'])

In [66]:
grouped_pct = grouped['tip_pct']

In [67]:
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

 Nếu khi bạn truyền 1 function hoặc tên của function , bạn sẽ lấy được 1 DataFrame với tên cột lấy từ functions : 

In [68]:
grouped_pct.agg(['mean','std'])

mean       std
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

Dưới đây tôi sẽ truyền vào 1 danh sách cho `aggregation()`to `agg()`

Nếu bạn truyền một danh sách của 1 tuple (name,func),('bả,function. Thì thử phần tử đầu tiên ủa mỗi tuple thường sử dụng như là 1 têm cột 
DataFrame 

In [69]:
grouped_pct.agg([('foo','mean'),('bar','std')])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

Với một DataFrame bạn có nhiều sự lựa chonh,  như bạn có thể chỉh sửa một danh sách của function đê ápp dụng tất cả các c

In [70]:
funcitons = ['count','mean','max']
result= grouped[['tip_pct','total_bill']].agg(funcitons)

In [71]:
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [72]:
ftuples = [('Durchschnitt','mean'),('Abweichung','var')]



In [73]:
grouped[['tip_pct','total_bill']].agg(ftuples)

tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

**NOTE**: Nó còn sử dụng chỉ định key cho các agg name fucntion bằng sử dụng `dict`

### Return Aggregated Data Without Row Indexes 

Trong tất cả các ví dụ gần đây đề sử dụng với index để `group` lại với nhau nhưng bây giờ có thể không cần sử dụng index như `as_index=False`]\

dưới đây là ví dụ k cần index để gọp lại chỉ cần bằng bởi Column

In [76]:
tips.groupby(['day','smoker'],as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


In [77]:
tips

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,No,Sat,Dinner,2,0.098204


### 10.3 Apply: General split-apply-combine

Phương pháp GroupBy có mục đích chung nhất được is `apply()`, là chủ đề của phần còn lại của phần này. hãy `apply() split() combine() ` các phép tách đối tượng đang được thao tác thành các mảnh, gọi hàm đã truyền trên mỗi mảnh và sau đó cố gắng nối các mảnh lại với nhau ”.

In [78]:
tips

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,No,Sat,Dinner,2,0.098204


In [79]:
def top(df,n=5,column='tip_pct'):
    return df.sort_values(by=column)[-n:]

In [84]:
top(tips,n=6)
#tips.sort_values(by='tip_pct')[-6:]

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


Bây giờ chúng ta có thể group `smoker`, và gọi `apply()` 
có nghĩa là bây giờ muốn group theo thằng tip_pct

In [85]:
tips.groupby('smoker').apply(top) 

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

Điều gì đã xảy ra ở trên , `top()` được gọi cho mỗi hàng từ DataFrame, và top sẽ gọi mỗi hàng từ DataFrame, và kết quả nó sẽ kết với nhau.

In [86]:
tips.groupby('smoker')

In [87]:
tips.groupby('smoker').count()

,total_bill,tip,day,time,size,tip_pct
smoker,,,,,,
No,151,151,151,151,151,151
Yes,93,93,93,93,93,93


Nếu bạn có thể truyền một hàm để `apply()` rằng nó có lấy được other arguments hoặc từ khoá, bạn có thể truyền nó sau hàm 


In [88]:
tips.groupby(['smoker','day']).apply(top,n=1,column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [89]:
result = tips.groupby('smoker')['tip_pct'].describe()

In [90]:
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


### Suppressing the Group Keys

In [91]:
tips.groupby('smoker',group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


### Example: Filling Missing values with Group Specific Values

Khi dọn dẹp **missing data**, trong một số trường hợp, bạn sẽ thay thế các quan sát dữ liệu bằng dropna, nhưng trong những trường hợp khác, bạn có thể muốn áp dụng (`fillna()`) các giá trị null (NA) bằng cách sử dụng một giá trị cố định hoặc một số giá trị bắt nguồn từ dữ liệu. fillna là công cụ phù hợp để sử dụng; ví dụ: ở đây tôi điền các giá trị NA với giá trị trung bình

In [92]:
s = pd.Series(np.random.randn(6))

In [93]:
s[::2] = np.nan

In [94]:
s

0         NaN
1    1.488765
2         NaN
3    0.998942
4         NaN
5   -0.259730
dtype: float64

In [95]:
s.fillna(s.mean())

0    0.742659
1    1.488765
2    0.742659
3    0.998942
4    0.742659
5   -0.259730
dtype: float64

Giả sử bạn cần giá trị điền thay đổi theo nhóm. Một cách để làm điều này là nhóm dữ liệu và sử dụng áp dụng với một hàm gọi fillna trên mỗi đoạn dữ liệu. Dưới đây là một số dữ liệu mẫu về các tiểu bang của Hoa Kỳ được chia thành các khu vực phía đông và phía tây: ”


In [101]:
states = ['Ohio','New York','Vermont','Florida','Oregon','Nevada','California','Idaho']

In [97]:
group_key = ['East']*4 + ['West']*4

In [98]:
group_key

['East', 'East', 'East', 'East', 'West', 'West', 'West', 'West']

In [102]:
data= pd.Series(np.random.randn(8),index=states)

In [103]:
data

Ohio          0.778066
New York      0.081276
Vermont       1.011898
Florida      -0.089454
Oregon       -0.839159
Nevada        0.405322
California   -0.067476
Idaho        -1.221522
dtype: float64

In [104]:
data[['Vermont','Nevada','Idaho']] = np.nan

In [105]:
data

Ohio          0.778066
New York      0.081276
Vermont            NaN
Florida      -0.089454
Oregon       -0.839159
Nevada             NaN
California   -0.067476
Idaho              NaN
dtype: float64

In [106]:
data.groupby(group_key).mean()

East    0.256629
West   -0.453317
dtype: float64

Chúng ta có thể điền các Na values sử dụng group means like so

In [107]:
fill_mean = lambda g: g.fillna(g.mean())

In [108]:
data.groupby(group_key).apply(fill_mean)

Ohio          0.778066
New York      0.081276
Vermont       0.256629
Florida      -0.089454
Oregon       -0.839159
Nevada       -0.453317
California   -0.067476
Idaho        -0.453317
dtype: float64

In [109]:
fill_values = {'East': 0.5,'West':-1}
fill_func = lambda g: g.fillna(fill_values[g.name])

In [110]:
data.groupby(group_key).apply(fill_func)

Ohio          0.778066
New York      0.081276
Vermont       0.500000
Florida      -0.089454
Oregon       -0.839159
Nevada       -1.000000
California   -0.067476
Idaho        -1.000000
dtype: float64